# MDA 2021
## Pyspark Sample Code
-----------------------------------------------------------------

## Setup
--------------------------------------------------

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
# !pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to processing data

**Make sure to follow the interactive instructions.**

In [2]:
# from google.colab import drive
# This will prompt for authorization.
# drive.mount('/content/drive')

Mounted at /content/drive


## Check and extract data
--------------------------------------------------

In [3]:
# !ls '/content/drive/MyDrive/SUT/Big Data/hw3'

In [4]:
file_path = "./"

In [2]:
!unzip './Sample_Data' -d './'

Archive:  ./Sample_Data.zip
  inflating: ./Sample_Traffic.csv    


the cells above, extract data which is in '/content/drive/My Drive/Test' to /content/drive/My Drive/Test/Traffic.csv  

## Initializing Spark and read data
--------------------------------------------------

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import col, current_timestamp, to_date, hour, dayofweek
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .config("spark.executor.cores", '6') \
    .config("spark.executor.memory", '4900m') \
    .getOrCreate()

sc = spark.sparkContext

schema = StructType([
    StructField("DEVICE_CODE", IntegerType(), True),
    StructField("SYSTEM_ID", IntegerType(), True),
    StructField("ORIGINE_CAR_KEY", StringType(), True),
    StructField("FINAL_CAR_KEY", StringType(), True),
    StructField("CHECK_STATUS_KEY", IntegerType(), True),
    StructField("COMPANY_ID", StringType(), True),
    StructField("PASS_DAY_TIME", TimestampType(), True)
])


<h2 dir="rtl">
 خواندن دیتا
</h2> 


In [5]:
df=spark.read.csv(f'{file_path}Sample_Traffic.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



In [6]:
df.head()

Row(DEVICE_CODE=200501, SYSTEM_ID=81, ORIGINE_CAR_KEY='10477885', FINAL_CAR_KEY='10477885', CHECK_STATUS_KEY=5, COMPANY_ID='161', PASS_DAY_TIME=datetime.datetime(2021, 6, 1, 3, 54, 39))

<h1 dir="rtl">
 الف 
</h1> 

<h2 dir="rtl">
 خواندن 
 <span dir="ltr">
   rdd 
 </span> 
  دیتافریم و کش کردن آن 
</h2> 


In [7]:
data = df.rdd
data.cache()

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

<h2 dir="rtl">
 بدست آوردن آیتم‌ها و سبد‌ها 
</h2> 


<h3 dir="rtl">
 آیتم‌ها 
</h3> 
<h4 dir="rtl">
 برای هر آیتم ما نیازمند پلاک شناسایی‌شده و روز به عنوان کلید و کد دوربین به عنوان مقدار آن نیازمندیم. باقی در این تمرین استفاده نمی‌شود. 
</h4> 
<h3 dir="rtl">
 سبد‌ها
</h3> 
<h4 dir="rtl">
ابتدا روی آیتم‌ها تابع 
<span dir="ltr">
  distinct
</span> 
را صدا می‌زنیم. با اینکار آیتم‌های یکتا را می‌گیریم و سپس آن‌هایی که برای یک ماشین در یک روز خاص هستند را یک گروه می‌کنیم و کد‌ دستگاه‌ها را ذخیره می‌کنیم.
</h4> 
<h3 dir="rtl">
  آیتم‌ها و سبد‌ها را کش می‌کنیم. دلیل استفاده از تابع 
  <span dir="ltr">
  distinct
</span> 
، برای از بین بردن تکرار برای یک کلید مشابه است.
به طور مثال ممکن است یک ماشین در یک روز خاص چندبار از یک‌محل رد شود.
</h3>


In [22]:
# (plate, date), route
car_route_per_day_rdd = data.map(lambda x: (
    (x.FINAL_CAR_KEY, x.PASS_DAY_TIME.date()), x.DEVICE_CODE))
car_route_per_day_rdd.cache()
# (plate, date), [route]
car_routes_list_per_day_rdd = car_route_per_day_rdd.distinct().groupByKey().map(lambda x: (x[0], tuple(x[1])))
car_routes_list_per_day_rdd.cache()


PythonRDD[56] at RDD at PythonRDD.scala:53

<h3 dir="rtl">
 به صورت رندم یک نمونه مسیر را می‌خوانیم و به عنوان یک مسیر برای آزمایش بخش‌های بعدی از آن استفاده می‌کنیم. 
</h3> 


In [14]:
# sample_path = car_routes_list_per_day_rdd.takeSample(False, 1, 5)[0][1]

In [28]:
car_routes_list_per_day_rdd.take(10)

[(('20428752', datetime.date(2021, 6, 1)), (230102,)),
 (('9261875', datetime.date(2021, 6, 1)),
  (200502,
   900222,
   900214,
   900102,
   900225,
   100700857,
   900227,
   100701130,
   631829,
   100700804,
   900167,
   631759,
   900271,
   209103)),
 (('88944712', datetime.date(2021, 6, 1)), (22010062,)),
 (('16533760', datetime.date(2021, 6, 1)),
  (22010062,
   100700862,
   900222,
   206601,
   900235,
   100700866,
   100700845,
   900149,
   900167)),
 (('7919894', datetime.date(2021, 6, 1)), (22010062,)),
 (('13776187', datetime.date(2021, 6, 1)), (22010062, 22010043)),
 (('78685492', datetime.date(2021, 6, 1)), (631830,)),
 (('12619573', datetime.date(2021, 6, 1)), (631830, 22010061)),
 (('78449990', datetime.date(2021, 6, 1)), (203902,)),
 (('7924039', datetime.date(2021, 6, 1)), (203902, 107301, 900191))]

<h3 dir="rtl">
 نمونه‌ای که بنده برای آزمایش انتخاب کرده‌ام. 
</h3> 


In [74]:
# sample_path = (203902, 107301, 900191)
sample_path = (22010062,
               100700862,
               900222,
               206601,
               900235,
               100700866,
               100700845,
               900149,
               900167)


<h2 dir="rtl">
 بدست آوردن لیست تمام کد دوربین‌ها 
</h2> 
<h3 dir="rtl">
 چون کد دوربین‌ها نشان‌دهنده یک مکان هستند و لیستی از آن‌ها مسیری را نشان می‌دهد؛ بنابراین برای درست کردن بردار‌هایی که می‌خواهیم برای مقایسه از آن‌ها استفاده کنیم، باید کد دوربین‌ها را داشته باشیم. 
</h3> 


In [12]:
device_codes = car_route_per_day_rdd.map(lambda x: x[1]).distinct().collect()


In [13]:
device_codes

[176,
 128,
 631784,
 168,
 631832,
 160,
 631488,
 631776,
 144,
 136,
 112,
 631800,
 631352,
 100701120,
 104,
 631368,
 22010048,
 22010080,
 22010040,
 22010088,
 900216,
 100701096,
 631640,
 100701144,
 900104,
 900152,
 22010112,
 120,
 232,
 22010120,
 900224,
 900256,
 900240,
 100700816,
 900272,
 900232,
 900208,
 631960,
 900184,
 100700880,
 631360,
 22010128,
 900264,
 1001016,
 22010072,
 605536,
 22010032,
 631888,
 900160,
 900176,
 900144,
 22009920,
 22009912,
 22009832,
 631608,
 635696,
 900120,
 22009808,
 22009960,
 22009840,
 635552,
 100700944,
 635608,
 100700928,
 22009952,
 1001024,
 100700960,
 100701160,
 900168,
 631928,
 1001040,
 631712,
 900136,
 210104,
 631560,
 631120,
 635616,
 22009928,
 711008,
 635568,
 631896,
 210216,
 635536,
 631664,
 210208,
 631848,
 100701224,
 631880,
 1001048,
 635704,
 631856,
 100700864,
 100700824,
 100700832,
 631984,
 635720,
 631696,
 631384,
 635712,
 100700920,
 100701136,
 22010136,
 22010096,
 100701264,
 635

<h1 dir="rtl">
 ب 
</h1> 
<h2 dir="rtl">
 مقایسه با شباهت کسینوسی

  
</h2> 


<h3 dir="rtl">
 تابع زیر برای محاسبه شباهت کسینوسی دو مسیر است. لیست دو مسیر را می‌گیرد و 
 سپس بردار‌های هرکدام را متشکل از یک سری مقدار یک اگر آن کد دوربین را در مسیرش دارد و یک سری مقدار صفر اگر آن کد دوربین در لیست مقابل وجود داشته‌باشد.
 سپس این دو بردار را به تابع کسینوس می‌دهیم تا محاسبه کند و سپس براساس مقداری که خروجی می‌دهد، آن‌ها را مرتب می‌کنیم و کمینه ترین مقدار‌ها را خروجی می‌دهیم.
</h3> 


In [30]:
from scipy.spatial.distance import cosine


def find_distance(a: list, b: list):
    a_dict = {}.fromkeys(a, 1)
    b_dict = {}.fromkeys(b, 1)
    all_codes = {}.fromkeys(a + b, 0)

    return cosine(list({**all_codes, **a_dict}.values()), list({**all_codes, **b_dict}.values()))


#  (plate, date), ([route], distance to sample_path)
car_routes_list_per_day_with_distance_rdd = car_routes_list_per_day_rdd.map(lambda x: (
    x[0], (x[1], find_distance(x[1], sample_path)))).sortBy(lambda x: x[1][1])


In [32]:
car_routes_list_per_day_with_distance_rdd.take(5)

[(('16533760', datetime.date(2021, 6, 1)),
  ((22010062,
    100700862,
    900222,
    206601,
    900235,
    100700866,
    100700845,
    900149,
    900167),
   0.0)),
 (('90782982', datetime.date(2021, 6, 1)),
  ((900222, 100700866, 900149, 100700845), 0.33333333333333337)),
 (('30345701', datetime.date(2021, 6, 1)),
  ((22010062, 900235, 100700845, 900167), 0.33333333333333337)),
 (('78742274', datetime.date(2021, 6, 1)),
  ((900167, 100700862, 100700845, 900235), 0.33333333333333337)),
 (('75968921', datetime.date(2021, 6, 1)),
  ((100700862, 100700845, 900149, 900235), 0.33333333333333337))]

<h1 dir="rtl">
 پ
</h1> 
<h2 dir="rtl">
مقایسه با 
<span dir="ltr">
 LSH 
</span> 
</h2>


<h3 dir="rtl">
 تعداد هش‌هایی که می‌خواهیم استفاده کنیم را تعیین می‌کنیم. 
</h3> 


In [50]:
min_hash_length = 30

<h3 dir="rtl">
 تابع 
 <span dir="ltr">
  create_min_hash 
 </span> 
 را اینگونه می‌سازیم که با احتمال مساوی به ازای هر کد دوربین ۱ یا -۱ نگه می‌داریم.
  
  سپس لیستی از هش‌ها به تعدادی که نیاز داریم می‌سازیم.
</h3> 


In [75]:
import random


def create_min_hash():
    min_hash = {}
    for code in device_codes:
        value = 1 if random.random() < 0.5 else -1
        min_hash[code] = value
    return min_hash


min_hash_list = [create_min_hash() for _ in range(min_hash_length)]


<h2 dir="rtl">
 اعمال هش برروی مسیر 
</h2> 
<h3 dir="rtl">
 به ازای کد‌هایی که در مسیر وجود دارد مقداری که برای آن کد در هش نگه داشتیم را جمع می‌کنیم. جمع این مقادیر یک مقدار جدیدی به ما می‌دهد که این مقدار را به
 عنوان مقدار اعمال‌شده این هش نگه می‌داریم. 
</h3> 


In [76]:
def aplly_min_hash(min_hash: dict, codes: list):
    value = 0
    for c in codes:
        value += min_hash[c]
    return value


<h2 dir="rtl">
 ساخت 
 <span dir="ltr">
  signature 
 </span> 
  هر مسیر
</h2> 

<h3 dir="rtl">
 با توجه به توضیحات بالا کافیست به ازای هر هش تابع 
 <span dir="ltr">
  aplly_min_hash 
 </span> 
  را صدا بزنیم و مقادیر را به صورت یک لیست ذخیره کنیم.
</h3> 


In [77]:
def create_signature(codes: list):
    signature = []
    for min_hash in min_hash_list:
        signature.append(aplly_min_hash(min_hash, codes))

    return signature


<h3 dir="rtl">
 حال تنها کار باقی‌مانده صدا زدن تابع 
 <span dir="ltr">
  create_signature 
 </span> 
  است. اینکار با یک تابع مپ انجام می‌دهیم.
</h3> 


In [78]:
# (plate, date), ([route], signature)
car_routes_signature_per_day_rdd = car_routes_list_per_day_rdd.map(
    lambda x: (x[0], (x[1], create_signature(x[1]))))
car_routes_signature_per_day_rdd.cache()


PythonRDD[117] at RDD at PythonRDD.scala:53

<h3 dir="rtl">
 از نمونه‌ای که با آن کار می‌کنیم، 
 <span dir="ltr">
  signature 
 </span> 
  را می‌سازیم.
</h3> 


In [79]:
sample_signature = create_signature(sample_path)


<h3 dir="rtl">
 حال باید هر امضا را به تکه‌هایی تقسیم کنیم. ابتدا تعداد نوار‌ها را مشخص می‌کنیم و با توجه به تعداد کل هش‌ها و تعداد نوار‌ها می‌توانیم سطر‌های هر نوار را بدست آوریم.
 تابعی می‌نویسیم تا هر لیست را به چند لیست براساس تعداد سطر‌ها و تعداد نوار‌ها تبدیل کند. 
</h3> 


In [80]:
from collections import defaultdict
band = 10
row = min_hash_length / band


def split_array(array: list):
    arrays = defaultdict(lambda: [])
    for index, a in zip(list(range(len(array))), array):
        arrays[int(index / row)].append(a)

    return list(arrays.values())


<h3 dir="rtl">
 با استفاده از این تابع نمونه خود را به چند لیست تبدیل می‌کنیم. 
</h3> 


In [81]:
split_sample_signature = split_array(sample_signature)

<h3 dir="rtl">
 حال هر مسیر را به چند تکه تقسیم می‌کنیم و هر تکه را متناظرا به تکه متناظر نمونه ست می‌کنیم و لیستی از دوتایی هایی که شامل تکه‌ای از نمونه و مسیر هستند داریم. سپس همه این دوتایی‌ها را چک می‌کنیم و سپس آن‌هایی که دوتایی یکسان دارند را نگه می‌داریم و آن مسیر‌ها را نگه می‌داریم. 
</h3> 


In [82]:
# flatMap: (plate, date, [route]), (split_sample_signature(a band),  split_signature(a band))
similar_car_routes_to_sample_signature = car_routes_signature_per_day_rdd.flatMap(lambda x: [((x[0][0], x[0][1], x[1][0]), (split_sample_signature[index], array))for index, array in zip(
    list(range(band)), split_array(x[1][1]))]).filter(lambda x: x[1][0] == x[1][1]).keys().distinct()


<h3 dir="rtl">
 سپس تمام این مسیر‌ها را نمایش می‌دهیم. 
</h3> 


In [83]:
similar_car_routes_to_sample_signature.map(lambda x: x[2]).distinct().collect()

[(22010031, 156, 900242),
 (22010062,
  100700862,
  900222,
  206601,
  900235,
  100700866,
  100700845,
  900149,
  900167)]

<h3 dir="rtl">
 باتوجه به آزمایشی که انجام دادم هرچه تعداد این هش‌ها زیاد شود، دقت تشابه نیز بالا می‌رود و نتایج مشابه‌تری را خروجی می‌دهد. 
</h3> 
